# Introduction
In this file, MVPA is applied on Z-stat files from first level analysis I have done on naive space, ART applied images, with each stimuli giving one Z-stat image.


    

In [1]:
import numpy as np
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
import itertools

import warnings
warnings.filterwarnings("ignore")
import os
import glob
from glob import glob
from scipy.io import loadmat
import math
import timeit
from timeit import default_timer as timer

plt.style.use('classic')
%matplotlib inline

In [8]:
from MVPA_func import SpherePoints, SphereValue, LoadAllNii, FeautrExtraction, Classification

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from sklearn.svm import LinearSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

### Load subject list file

In [4]:
subj_list_file='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/subj_simultaneous.txt'
subj_list=list()
f = open(subj_list_file, 'r')
line=f.readline()
while line:
    subj_list.append(line[0:-1])
    oldline=line
    line = f.readline()
subj_list.pop()
subj_list.append(oldline)
f.close()
Nsubj=len(subj_list)

### Load SPM.mat to extract experiment info
This cell loads spm.mat file for each participant and from the data, extracts the number of runs, the path for each stat file, etc

In [5]:
# note: I don't know why the last item is repeated! but it doesn't make a misatke

conditions=['Neu','Neg','Pos','Targ']
ZstatInfo=pd.DataFrame(columns=['Subj','Run','Folder']+conditions)
for subj in subj_list:
    print('Participant: '+subj)
    # Zstat files and their condisitons are in SPM filesWe repeat loading in the dataframe to avoid dividing the onsets by 2 multiple times ...
    SubjSPMFolder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/'+subj+'/3cond_dist2/'
    os.path.isdir(SubjSPMFolder)
    myZstat=[]
    
    nRuns = len(os.listdir(SubjSPMFolder))
    iRuns = 0
    for myRun in os.listdir(SubjSPMFolder):
        ZstatInfotmp=pd.DataFrame(columns=ZstatInfo.columns,index=[0])
        ZstatInfotmp['Subj'][0]=subj
        
        RunSubjSPM=SubjSPMFolder+myRun+'/SPM.mat'
        ZstatInfotmp['Folder'][0]=SubjSPMFolder+myRun+'/'

        print('   loading SPM.mat for '+myRun+'...')
        mat = loadmat(RunSubjSPM)
        mdata=mat['SPM'][0,0]
        ndata=mdata['Sess'][0,0]
        odata=ndata['U']
        pdata=odata['name'][0]
        ZstatNames=[pdata[i][0,0][0] for i in range(len(pdata))]
        myZstat.append(ZstatNames)

        ZstatInfotmp['Run'][0]=iRuns+1
        old_ind=len(myZstat[0])+1
        for iCond in reversed(conditions):    
            Start_ind=myZstat[0].index(iCond+'_1')+1
            #print(Start_ind)
            ZstatInfotmp[iCond][0]=['spmT_'+str(val).zfill(4)+'.nii' for val in list(range(Start_ind, old_ind))]#list(range(Start_ind, old_ind))
            old_ind=Start_ind
        ZstatInfo=ZstatInfo.append(ZstatInfotmp,ignore_index=True)
        iRuns = iRuns+1
 

Participant: AB_04152014
   loading SPM.mat for run01...
   loading SPM.mat for run02...
   loading SPM.mat for run03...
   loading SPM.mat for run04...
   loading SPM.mat for run05...
Participant: AM_05082014
   loading SPM.mat for run01...
   loading SPM.mat for run02...
   loading SPM.mat for run03...
   loading SPM.mat for run04...
   loading SPM.mat for run05...
Participant: CJ_06052014
   loading SPM.mat for run01...
   loading SPM.mat for run02...
   loading SPM.mat for run03...
   loading SPM.mat for run04...
   loading SPM.mat for run05...
Participant: CS_04162014
   loading SPM.mat for run01...
   loading SPM.mat for run02...
   loading SPM.mat for run03...
   loading SPM.mat for run04...
   loading SPM.mat for run05...
Participant: DH_05142014
   loading SPM.mat for run01...
   loading SPM.mat for run02...
   loading SPM.mat for run03...
   loading SPM.mat for run04...
   loading SPM.mat for run05...
Participant: DS_12132013
   loading SPM.mat for run01...
   loading SPM.mat

### find Voxel dimension by openning one of the image files


In [6]:
MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']==subj) & (ZstatInfo['Run']==1)]
MyZstat.reset_index(inplace = True, drop = True)
MyFile=MyZstat['Folder'][0]+MyZstat['Neu'][0][0]
print(MyFile)
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)


C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/TP_05162014/3cond_dist2/run01/spmT_0001.nii
(64, 64, 28)


If it is needed to modify a funciton and relaod it:

In [12]:
from importlib import reload #import function "reload"
import sys
SpherePoints = reload(sys.modules["MVPA_func"]).SpherePoints  # reload() returns the new module
#from MVPA_func reload SpherePoints

In [13]:
start = timer()

radius = 1 #spheres have radius 2
tmpPoints = SpherePoints(np.array([26,26,16]) , radius, voxel_dims)


In [14]:
# Calculate the spotlights voxels for all points and keep them in AllPoints 
start = timer()

radius = 1 #spheres have radius 2
tmpPoints = SpherePoints(np.array([26,26,16]) , radius, voxel_dims)
AllZeroSphere = np.zeros((1,tmpPoints.shape[2]))
AllZeroSpherePoints=np.zeros(tmpPoints.shape)
cnt=0
for MyCenter in itertools.product(range(0,voxel_dims[0]), range(0,voxel_dims[1]), range(0,voxel_dims[2])):
    #print(MyCenter)
    myPoints = SpherePoints(np.array(MyCenter) , radius, voxel_dims)
    if myPoints.shape==(3,1,0): myPoints=AllZeroSpherePoints
    if cnt==0:
        AllPoints = np.array(myPoints,ndmin=4)
        cnt=1
    else:
        AllPoints = np.concatenate((AllPoints,np.array(myPoints,ndmin=4)),axis=0)
#     if myPoints.shape!=(3,1,0): sphereVal = data[myPoints[0],myPoints[1],myPoints[2]]
#     else: sphereVal = AllZeroSphere
        
#    if myPoints.shape!=(0,): sphereVal = SphereValue(myPoints,data)
#     if sphereVal.shape!=(0,): sphereValOut=


end = timer()
print(end-start)

308.3831950998865


In [16]:
# This code does classification based on spot light search
start = timer()
CLconditions=['Neu','Neg','Targ']
radius = 1 #spheres radius


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

subj_list_one=[subj_list[1]]; 
Nsubj=len(subj_list_one)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list_one):
    
    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    
    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
     
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    for iFeature in range(56500,56510):#range(AllPoints.shape[0]): #iFeature=40836
        myPoints = AllPoints[iFeature] 
        # myPoints = SpherePoints(np.array([30,30,15]) , radius) # sample good point

        [X, Y, Gr]=FeautrExtraction(ZstatInfoSubj,CLconditions,Alldata, myPoints) # extract data for classifer
        SpotPerformance=Classification(X,Y,Gr, pipeSpotLight)

        SpotLight_performance[iFeature]= SpotPerformance
        

end = timer()
print(end-start)
SpotLight_performance[56500:56510]

NameError: name 'GroupKFold' is not defined

In [12]:
##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

#pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
#pipe.fit(X_train, Y_train)
pipeSpotLight.fit(X_train, Y_train)
preds = pipeSpotLight.predict(X_train)
pipeSpotLight

NameError: name 'X_train' is not defined

In [267]:
for i in range(3,6):
    print(i)
iFeature=56500
AllPoints[iFeature] 


3
4
5


array([[[30., 31., 31., 31., 31., 31., 32.]],

       [[33., 32., 33., 33., 33., 34., 33.]],

       [[24., 24., 23., 24., 25., 24., 24.]]])

In [289]:
SpotLight_performance[56500:56510]

array([0.503125  , 0.58993056, 0.47204861, 0.        , 0.        ,
       0.41822917, 0.56614583, 0.49392361, 0.62256944, 0.58732639])

In [290]:
tmp

array([0.48888889, 0.57986111, 0.45486111, 0.        , 0.        ,
       0.47638889, 0.55972222, 0.50329861, 0.625     , 0.58854167])

In [25]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, preds,target_names=CLconditions))

              precision    recall  f1-score   support

         Neu       0.55      0.75      0.63         8
         Neg       0.33      0.12      0.18         8
        Targ       0.86      1.00      0.92        12

    accuracy                           0.68        28
   macro avg       0.58      0.62      0.58        28
weighted avg       0.62      0.68      0.63        28



This code generates sphere around around a center and predefined sphere

In [22]:
from sklearn.metrics import confusion_matrix
print(accuracy_score(Y_test, preds))
confusion_matrix(Y_test, preds)

array([[ 6,  2,  0],
       [ 5,  1,  2],
       [ 0,  0, 12]], dtype=int64)

In [9]:
Mypreds=[2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 2, 1, 3]# 4th fold
Mypreds=preds#5th fold
print(Mypreds)
print(Y_test)
print("Accuracy test: %.2f" % (Mypreds == Y_test).mean())
Y12, P12 =Y_test[0:16], Mypreds[0:16]
Y13, P13=np.concatenate((Y_test[0:8],Y_test[16:])), np.concatenate((Mypreds[0:8],Mypreds[16:]))
Y23, P23=Y_test[8:], Mypreds[8:]
ROC12=roc_auc_score(Y12,P12)
ROC13=roc_auc_score(Y13,P13)
ROC23=roc_auc_score(Y23,P23)

print('Pairwise AUC: ',ROC12,ROC13,ROC23)
print('pairwise AUC averaged mean:',ROC12*16/56+ROC13*20/56+ROC23*20/56)
performance_this_participant

[1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 3. 2. 1. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]
[1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]
Accuracy test: 0.68
Pairwise AUC:  0.59375 1.0 0.875
pairwise AUC mean 0.8392857142857143


array([0.80989583, 0.87326389, 0.81336806, 0.81857639, 0.73871528])

In [10]:
print(Y13)
print(P13)
ROC13=roc_auc_score(Y13,P13)
print(ROC13)


[1. 1. 1. 1. 1. 1. 1. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
[1. 1. 1. 2. 1. 1. 1. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
1.0


In [168]:
select100best = SelectKBest(f_classif, k=100)

##
#clf=MultiOutputClassifier(SVC(kernel='linear'))
clf = SVC(kernel='linear',probability=True)
pipe = Pipeline([('scaler', StandardScaler()), ('ufs', select100best), ('clf', clf)])
pipe.fit(X_train, Y_train)
pipe.decision_function(X_test)
pipe.predict_proba(X_test)
# roc_auc_score(Y_test, pipe.predict_proba(X_test),  multi_class='ovr')
roc_auc_score(Y_test, pipe.predict_proba(X_test),  multi_class='ovo')

#clf.predict_proba(X)

0.7196180555555555

In [167]:
print('Correct labels:',Y_test)
print('Predicted labels:',preds)
print("Accuracy test: %.2f" % (preds == Y_test).mean())

Correct labels: [1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]
Predicted labels: [1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 3. 2. 1. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]
Accuracy test: 0.68


In [138]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(solver="liblinear").fit(X, y)
yt=clf.predict_proba(X)
roc_auc_score(y, yt, multi_class='ovr')
np.sum(clf.predict_proba(X), axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [303]:
# Try out different estimators below (call fit and predict on X and y!)
lda = LinearDiscriminantAnalysis()
lda.fit(X, Y)
y_hat=lda.predict(X)
print('accuracty: '+ str(accuracy_score(Y, y_hat)))


accuracty: 0.6571428571428571


In [61]:
# old file, no need to run it
# Calculate the spotlights voxels for all feasible voxel. check which one is faster
start = timer()
iSpot=0
#for MyCenter in itertools.product(range(radius,voxel_dims[0]-radius), range(radius,voxel_dims[1]-radius), range(radius,voxel_dims[2]-radius)):
for MyCenter in itertools.product(range(0,voxel_dims[0]), range(0,voxel_dims[1]), range(0,voxel_dims[2])):
    #print(MyCenter)
    myPoints = SpherePoints(np.array(MyCenter) , radius)
    if myPoints.shape==(3,1,0): myPoints=AllZeroSpherePoints
    if iSpot==0:
        SpotPoints=np.array([myPoints])
        iSpot=1
    else:
        SpotPoints=np.append(SpotPoints,np.array([myPoints]), axis=0)
    

#voxel_dimCut=[voxel_dims[0]-2*radius,voxel_dims[1]-2*radius,voxel_dims[2]-2*radius]
# for idata in range(X.shape[0]):
#     dataN=X[idata].reshape(voxel_dims,order='C')
#     if myPoints.shape!=(3,1,0): sphereVal = dataN[myPoints[0],myPoints[1],myPoints[2]]
end = timer()
print(end-start)